In [ ]:
import medspacy
from IPython.display import Image

In [ ]:
from medspacy.visualization import visualize_dep, visualize_ent
from helpers import MedspaCyVisualizerWidget

# Overview
In the last notebook, we used a statistical NLP model to extract clinical events such as problems, treatments, and tests. However, just because a report mentions a clinical concept doesn't mean that a patient actually has that concept.

Another important task in clinical NLP is **contextual analysis**, which involves looking for contextual modifiers around a concept which indicate whether a concept is:
- Negated
- Historical
- Uncertain
- Experienced by someone other than the patient (such as family history)
- Hypothetical (something that could occur in the future)

## The ConText algorithm
One method for performing this analysis is the **ConText** algorithm. This algorithm was originally proposed in this paper: [Context: An Algorithm for Determining Negation, Experiencer, and Temporal Status from Clinical Reports](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2757457/) by Harkema et al. ConText is an extension of the NegEx algorithm, which is very similar.

There are several implementations of ConText and clinical NLP systems which use ConText, including:
- [cTAKES](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2995668/)
- [Leo](https://department-of-veterans-affairs.github.io/Leo/index.html)
- [pyConText](https://github.com/chapmanbe/pyConTextNLP)

## How ConText works

ConText connects certain **modifiers**, such as **"no evidence of"** or no **"is negative"**, with the target concepts we are extracting. 

---
There is **no evidence of** **_pneumonia_**

---
In this sentence, the **target** is **_pneumonia_**: this is the clinical concept we are trying to extract. The **modifier** is **no evidence of**: this shows that the concept is **negated**. 

ConText finds these targets and modifiers in text and builds **relationships** between them. Here is a visual representation of the graph ConText would create out of this sentence:


In [ ]:
Image("./images/negated_pneumonia.png", width=600)

Likewise, here is another example using family history:

---
There is **_diabetes_** **on her mother's side**

---

In [ ]:
Image("./images/family_history_diabetes.png", width=600)

In both of these examples, finding the clinical problem in the text is not enough: you also need to recognize that the concept is negated or in the patient's family, rather than the patient themself.

# ConText with medspaCy
We will use context by adding it to our NLP pipeline, just like we did with the **EntityRuler**. First, let's load our model in which contains our NER trained to extracted clinical concepts:

In [ ]:
nlp = medspacy.load("en_info_3700_i2b2_2012")

We can access the ConText component in the pipeline:

In [ ]:
context = nlp.get_pipe("context")

In [ ]:
context

In [ ]:
context.categories

## A simple example
Let's go back to the example we saw in our last notebook. Our NER model correctly identified **"pneumonia"** as a **problem**, but in this sentence it is explicitly negated:

---
There is **no evidence of** **_pneumonia_**

---

Using the ConText algorithm, we can now recognize that this concept is negated. Thanks to the modular nature of spaCy processing pipelines, we don't need to do anything different:

In [ ]:
doc = nlp("There is no evidence of pneumonia.")

Let's visualize what the ConText algorithm is doing. MedspaCy offers two functions for visualizing the algorithm. The first function, `visualize_ent`, visualizes the clinical concepts and modifiers in an NER-style visualization, as we saw in the last notebook:

In [ ]:
from medspacy.visualization import visualize_dep, visualize_ent

In [ ]:
visualize_ent(doc)

The second, `visualize_dep` one visualizes the relationships between targets and modifiers in a dependency-style visualization. Here, we can see that the modifier **"no evidence of"** is applied to the target **"pneumonia"**.

In [ ]:
visualize_dep(doc)

When an entity is negated by context, the negation is stored in the `ent._.is_negated` attribute. If `True`, then the concept is negated. By default, it will be `False`.

In [ ]:
for ent in doc.ents:
    print(ent, ent._.is_negated)

## Other attributes
By default, medspaCy will extract the following attributes, which are all False unless the entity is modified by a certain type of modifier:
- `ent._.is_negated`
- `ent._.is_historical`
- `ent._.is_uncertain`
- `ent._.is_family`
- `ent._.is_hypothetical`

Let's see some more examples:

### Historical

In [ ]:
doc = nlp("Past medical history significant for nephrectomy.")

In [ ]:
visualize_dep(doc)

In [ ]:
for ent in doc.ents:
    print(ent, "Historical:", ent._.is_historical)

### Uncertainty

In [ ]:
doc = nlp("The scan likely shows a pneumothorax.")

In [ ]:
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

In [ ]:
for ent in doc.ents:
    print(ent, "Uncertain:", ent._.is_uncertain)

### Family history

In [ ]:
doc = nlp("Her mother had breast cancer.")

In [ ]:
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

In [ ]:
for ent in doc.ents:
    print(ent, "Family:", ent._.is_family)

### Hypothetical

In [ ]:
doc = nlp("She should stop taking warfarin if she develops a rash.")

In [ ]:
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

In [ ]:
for ent in doc.ents:
    print(ent, "Hypothetical:", ent._.is_hypothetical)

# Creating your own modifiers
We've been using the default knowledge base that comes with context. However, you may want to modify or context's behavior, including adding brand new concepts.

Let's replace our **context** component with a blank instance, then add our own rules.

In [ ]:
from medspacy.context import ConTextComponent, ConTextItem

In [ ]:
blank_context = ConTextComponent(nlp, rules=None);

In [ ]:
# Now replace our old context component with this
if "context" in nlp.pipe_names:
    nlp.remove_pipe("context")
nlp.add_pipe(blank_context)

Let's go back to our first example.

In [ ]:
text = "There is no evidence of pneumonia."

## ConTextItem
The modifier rules in context are controlled by `ConTextItem`. A ConTextItem defines what span of text to match as a modifier, how that modifier behaves, and the semantic category of the modifier. It takes these main arguments:
- **`literal`**: The exact text to match
- **`category`**: The semantic category of the modifier, such as **"NEGATED_EXISTENCE"** or **"HISTORICAL"**
- **`rule`**: Which **direction** the modifier should look in the sentence. 
    - Look back at the two images at the top of the notebook. 
    - In the first example, "There is **no evidence** of **_pneumonia_**", the modifier is **"no evidence"** and it comes before the target concept. In that case, we say it moves **"forward"** in the sentence (to the right). 
    - In the other example, "There is **_diabetes_** **on her mother's side**", the modifier comes after the target and we say it moves **backward** in the sentence. 
    - This argument in context can take the following values:
        - **"BIDIRECTIONAL"** - This is the default and the modifier will apply to targets on both sides of the modifier
        - **"FORWARD"** - The modifier will modify any targets *after* the modifier
        - **"BACKWARD"** - The modifier will modify any targets *before* the modifier
        - **TERMINATE"** - Any modifiers will stop at this point, such as **"but"** in "No evidence of pneumonia or pleural opacities **but** he has PE."
- **`pattern`**: An optional spaCy pattern to match, like we saw in the pattern-matching notebooks

Let's import the `ConTextItem` from medspaCy.

In [ ]:
from medspacy.context import ConTextItem

## Example 1: Negation
Let's create a `ConTextItem` to negate **"pneumonia"** in our first example: "There is **no evidence** of **_pneumonia_**".

### TODO
Create a `ConTextItem` with the following arguments:
- **"no evidence of"**: This will match the phrase in the text
- **"NEGATED_EXISTENCE"**: This is the semantic category
- **"FORWARD"**: The target concept comes *after* the modifier in the sentence

In [ ]:
item = ConTextItem(____, category=____, rule=____)

We then add a list of ConTextItems to our context object:

In [ ]:
blank_context.add([item])

In [ ]:
blank_context.item_data

Now when we call `nlp` on our text, we can see that **"pneumonia"** is negated by the modifier.

In [ ]:
doc = nlp("There is no evidence of pneumonia.")

In [ ]:
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

## Example 2: Family History

Now let's identify the **family** modifier in "There is diabetes on her mother's side."

### TODO
Create a ConTextItem which will match **"mother's side"** and modify **"diabetes"**. It should have the category **"FAMILY"** and the rule should be **"BACKWARD"**.

In [ ]:
doc = nlp("There is diabetes on her mother's side.")
doc.ents

In [ ]:
item = ConTextItem(____, category=____, rule=____)

In [ ]:
blank_context.add([item])

In [ ]:
doc = nlp("There is diabetes on her mother's side.")
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

## Example 3: Uncertainty
In the phrase below, the physician is considering both **"pneumonia"** and **"bronchitis"** as a diagnosis. In this case, the modifier should go in both directions, not just **"forward"** or **"backward"**.

### TODO
Create a ConTextItem which matches both targets. The category should be **"POSSIBLE_EXISTENCE"** and the rule should be **"BIDIRECTIONAL"**.

In [ ]:
doc = nlp("Pneumonia vs bronchitis")
doc.ents

In [ ]:
item = ConTextItem(____, category=____, rule=____)

In [ ]:
blank_context.add([item])

In [ ]:
doc = nlp("Pneumonia vs bronchitis")
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

# Additional examples
Below are a number of additional texts. Go through each of them and process with the NLP. Identify which modifiers should be matched in the sentence and create ConTextItems to connect the modifiers with the targets.

The **category** arguments in the ConTextItems can be:
- "NEGATED_EXISTENCE"
- "POSSIBLE_EXISTENCE"
- "HISTORICAL"
- "HYPOTHETICAL"
- "FAMILY"

And the **rule** arguments can be:
- "BIDIRECTIONAL"
- "FORWARD"
- "BACKWARD"
- "TERMINATE"

In [ ]:
item_data = [
    ConTextItem(____, category=____, rule=____),
    # Add additional context rules here
]

In [ ]:
blank_context.add(item_data)

In [ ]:
texts = [
    "His wife recently died from end stage renal disease.",
    "Whether this is pneumonia is unknown.",
    "Pneumonia vs. bronchitis",
    "Past medical history significant for afib, CHF, and CKD.",
    "Pt's grandfather had prostate cancer.",
    "Stop taking medications if any side effects occur.",
    "The respiratory panel returned negative for influenza.",
    
]

In [ ]:
docs = list(nlp.pipe(texts))

To see the relationships in docs, try setting the widget visualization mode to `Dep` or `Both`.

In [ ]:
w = MedspaCyVisualizerWidget(docs)

In [ ]:
# idx = 0
# visualize_dep(docs[idx])
# visualize_ent(docs[idx])

# Next Steps
We now have a fairly comprehensive set of tools for processing clinical text:
1. A pre-trained statistical model which can detect clinical problems, treatments, and tests
2. A rule-based matcher which can extract additional entities which are not extracted by our NER
3. ConText to detect attributes such as negation, temporality, and uncertainty

In our next notebook, we'll put all of this together to analyze an **annotated dataset** and evaluate how well our system works on MIMIC data.


[05-clinical_information_extraction.ipynb](05-clinical_information_extraction.ipynb)